## W&Bでの埋め込みの可視化

データを[Weights & Biases](http://wandb.ai)にアップロードし、[Embedding Projector](https://docs.wandb.ai/ref/app/features/panels/weave/embedding-projector)を使用して、PCA、UMAP、t-SNEなどの一般的な次元削減アルゴリズムで埋め込みを可視化します。データセットは[Get_embeddings_from_dataset Notebook](Get_embeddings_from_dataset.ipynb)で作成されています。

## Weights & Biasesとは？

[Weights & Biases](http://wandb.ai)は、OpenAIや他の機械学習チームがより良いモデルをより速く構築するために使用している機械学習プラットフォームです。実験の迅速な追跡、モデル性能の評価、モデルの再現、結果の可視化、そして同僚との知見の共有に使用されています。

### 1. データをW&Bにログする

元のデータと埋め込みベクトルを含む[W&B Table](https://docs.wandb.ai/guides/data-vis/log-tables)を作成します。各レビューは新しい行となり、1536個の埋め込みベクトルの浮動小数点数は、それぞれ`emb_{i}`という名前の独自の列に格納されます。

In [2]:
import pandas as pd
from sklearn.manifold import TSNE
import numpy as np
from ast import literal_eval

# Load the embeddings
datafile_path = "data/fine_food_reviews_with_embeddings_1k.csv"
df = pd.read_csv(datafile_path)

# Convert to a list of lists of floats
matrix = np.array(df.embedding.apply(literal_eval).to_list())

In [ ]:
import wandb

original_cols = df.columns[1:-1].tolist()
embedding_cols = ['emb_'+str(idx) for idx in range(len(matrix[0]))]
table_cols = original_cols + embedding_cols

with wandb.init(project='openai_embeddings'):
    table = wandb.Table(columns=table_cols)
    for i, row in enumerate(df.to_dict(orient="records")):
        original_data = [row[col_name] for col_name in original_cols]
        embedding_data = matrix[i].tolist()
        table.add_data(*(original_data + embedding_data))
    wandb.log({'openai_embedding_table': table})

### 2. 2D投影としてレンダリング

W&Bの実行リンクに移動した後、テーブルの右上にある⚙️アイコンをクリックし、「Render As:」を「Combined 2D Projection」に変更します。

例: http://wandb.me/openai_embeddings